In [ ]:
import random
import math
import pandas as pd


num_flights = 20
num_classA_pilots = 25
num_classB_pilots = 25
num_classA_copilots = 25
num_classB_copilots = 25
num_cabin_crew = 100

# Generate random flight schedule data and fixed operation costs
flights = [f"Flight-{i+1}" for i in range(num_flights)]

boeing_flights = random.sample(flights, random.randint(5, 20))
for flight in boeing_flights:
    flights.remove(flight)
    flights.append(f"Boeing-{flight.split('-')[1]}")
random.shuffle(flights)

operation_costs = {flight: random.randint(5000, 10000) for flight in flights}

cost_per_pilot = 2000
cost_per_copilot = 1500
cost_per_cabin_crew = 1000

# Crew Assignment
def generate_crew_assignments_one_pilot():
    crew_assignments = {}
    pilots_assigned = []

    for flight in flights:
        num_copilots_assigned = random.randint(1, 2)
        num_cabin_crew_assigned = random.randint(1, 5)

        if 'Boeing' in flight:  # Class B
            copilots = random.sample(range(num_classA_copilots + 1, num_classA_copilots + num_classB_copilots + 1), num_copilots_assigned)
        else:  # Class A
            copilots = random.sample(range(1, num_classA_copilots + 1), num_copilots_assigned)


        pilot = random.randint(1, num_classA_pilots + num_classB_pilots)
        while pilot in pilots_assigned:
            pilot = random.randint(1, num_classA_pilots + num_classB_pilots)
        pilots_assigned.append(pilot)

        cabin_crew = random.sample(range(1, num_cabin_crew + 1), num_cabin_crew_assigned)

        crew_assignments[flight] = {'Pilots': [pilot], 'CoPilots': copilots, 'CabinCrew': cabin_crew}
    return crew_assignments

# Function to calculate cost
def calculate_cost(crew_assignments):
    total_cost = 0
    for flight, assigned_crews in crew_assignments.items():
        flight_cost = operation_costs[flight]

        pilot_cost = len(assigned_crews['Pilots']) * cost_per_pilot
        copilot_cost = len(assigned_crews['CoPilots']) * cost_per_copilot
        cabin_crew_cost = len(assigned_crews['CabinCrew']) * cost_per_cabin_crew

        total_cost += flight_cost + pilot_cost + copilot_cost + cabin_crew_cost
    return total_cost

# Function to perturb
def perturb_solution(current_solution):
    perturbed_solution = current_solution.copy()

    flight_to_modify = random.choice(flights)
    current_crews = perturbed_solution[flight_to_modify]

    current_crews['Pilots'] = random.sample(range(1, num_classA_pilots + 1), 1) if 'Airbus' in flight_to_modify else random.sample(range(num_classA_pilots + 1, num_classA_pilots + num_classB_pilots + 1), 1)
    current_crews['CoPilots'] = random.sample(range(1, num_classA_copilots + 1), random.randint(1, 2)) if 'Airbus' in flight_to_modify else random.sample(range(num_classA_copilots + 1, num_classA_copilots + num_classB_copilots + 1), random.randint(1, 2))
    current_crews['CabinCrew'] = random.sample(range(1, num_cabin_crew + 1), random.randint(1, 5))

    return perturbed_solution

# Function for Simulated Annealing
def simulated_annealing(initial_solution, calculate_cost, perturb_solution, temperature):
    current_solution = initial_solution
    best_solution = initial_solution
    current_cost = calculate_cost(current_solution)
    best_cost = current_cost

    while temperature > 0.1:
        new_solution = perturb_solution(current_solution)
        new_cost = calculate_cost(new_solution)

        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost

        if new_cost < best_cost:
            best_solution = new_solution
            best_cost = new_cost

        temperature *= 0.99

    return best_solution, best_cost


initial_solution_one_pilot = generate_crew_assignments_one_pilot()


optimal_solution_one_pilot, optimal_cost_one_pilot = simulated_annealing(initial_solution_one_pilot, calculate_cost, perturb_solution, 1000)

# Output
print("Optimal Crew Assignments:")
for flight, assigned_crews in optimal_solution_one_pilot.items():
    flight_id = flight if 'Boeing' not in flight else f"Flight-{flight.split('-')[1]}"
    flight_cost = operation_costs[flight] \
                    + len(assigned_crews['Pilots']) * cost_per_pilot \
                    + len(assigned_crews['CoPilots']) * cost_per_copilot \
                    + len(assigned_crews['CabinCrew']) * cost_per_cabin_crew

    flight_class = "Boeing" if 'Boeing' in flight else "Airbus"
    print(f"Flight ID: {flight_id}, Class: {flight_class}, Assigned Crew: {assigned_crews}, Cost: {flight_cost}")


data = []
for flight, assigned_crews in optimal_solution_one_pilot.items():
    pilots = assigned_crews['Pilots'][0] if assigned_crews['Pilots'] else None
    copilots = assigned_crews['CoPilots']
    cabin_crew = assigned_crews['CabinCrew']
    data.append({'Flight_ID': flight, 'Pilots': pilots, 'CoPilots': copilots, 'CabinCrew': cabin_crew})

assignments_df_one_pilot = pd.DataFrame(data)
assignments_df_one_pilot
